# IMPC Knockout Mouse Phenotypes
This notebook contains the processing scripts for the IMPC Knockout Mouse Phenotypes dataset. Data was downloaded from the [IMPC FTP site](http://ftp.ebi.ac.uk/pub/databases/impc/), where the Phenotype Hits per Gene file was converted to a gene-phenotype edgelist of 6763 genes and 667 phenotypes.

In [ ]:
import pandas as pd
import datetime
import numpy as np
import scipy.spatial.distance as dist
import seaborn as sns
import sys
import json
import scanpy as sc
from tqdm import tqdm

# UMAP
from sklearn.feature_extraction.text import TfidfVectorizer
import anndata
from collections import OrderedDict

# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show, save, output_file
from bokeh.models import HoverTool, ColumnDataSource
from bokeh.palettes import Category20
output_notebook()

from IPython.display import display, HTML, Markdown
sys.setrecursionlimit(100000)

## Load and Pre-Process Data

In [ ]:
komp = pd.read_csv('phenotypeHitsPerGene.csv').dropna(subset='Phenotype Hits')
komp

In [ ]:
komp['Phenotype Hits'] = komp['Phenotype Hits'].apply(str.split, sep='::')
komp = komp.explode('Phenotype Hits').get(['Gene Symbol', 'Phenotype Hits'])
komp

In [ ]:
len(komp['Gene Symbol'].unique()), len(komp['Phenotype Hits'].unique())

### Map Gene Symbols to Up-to-date and Approved Symbols and NCBI Gene IDs

In [ ]:
genemapping = pd.read_csv('../../tables/MouseHumanGeneMapping.txt', sep='\t', header=None).drop(columns=[4,5])
def mapmousegenes(accessionid):
    if accessionid in genemappingdict:
        return genemappingdict[accessionid]
    return np.nan

genemappingdict = genemapping.set_index(2)[0].to_dict()
komp['Gene Symbol'] = komp['Gene Symbol'].apply(mapmousegenes)
komp = komp.dropna()
komp

In [ ]:
len(komp['Gene Symbol'].unique()), len(komp['Phenotype Hits'].unique())

In [ ]:
def mapupdategenes (gene):
    if gene in genemappingdict:
        return genemappingdict[gene]
    return np.nan

genemappingdict = pd.read_csv('../../mapping/mappingFile_2023.tsv', sep='\t', header=None).set_index(1)[2].to_dict()
komp['Gene Symbol'] = komp['Gene Symbol'].apply(mapupdategenes)
komp = komp.dropna().drop_duplicates()
komp

### Remove Obsolete Terms and Phenotypes Without MPO Mapping

In [ ]:
phenotypes = pd.read_csv('MGIMammalianPhenotype.txt', sep='\t', header=None, index_col=1)
komp = komp[komp['Phenotype Hits'].apply(lambda x: x in phenotypes.index)]
komp = komp[komp['Phenotype Hits'].apply(lambda x: 'obsolete' not in x)]
#komp['Phenotype ID'] = komp['Phenotype Hits'].apply(lambda x: phenotypes.loc[x, 0])
komp = komp.reset_index(drop=True)
komp

In [ ]:
len(komp['Gene Symbol'].unique()), len(komp['Phenotype Hits'].unique())

## Process Data for SQL

### Resource

In [ ]:
#(id, name, acronym, long_description, short_description, url, num_attributes, num_datasets)
(109, 'International Mouse Phenotyping Consortium', 'IMPC', 'The International Mouse Phenotyping Consortium (IMPC) is an international effort by 21 research institutions to identify the function of every protein-coding gene in the mouse genome.', 'A comprehensive catalogue of mammalian gene information', 'https://mousephenotype.org', 670, 1)

### Dataset

In [ ]:
#(id, name, name_without_resource, description, association, gene_set_description, gene_sets_description, attribute_set_description, is_signed, is_continuous_valued, last_updated, directory, num_page_views, resource_fk, measurement_fk, dataset_group_fk, attribute_type_fk, attribute_group_fk, evidence_type, evidence_group, measurement_bias, attribute_type_plural, is_archived)
(148, 'IMPC Knockout Mouse Phenotypes', 'Knockout Mouse Phenotypes', 'observed phenotypes of mice following gene knockout', 'gene-phenotype associations from phenotyping studies of knockout mice', 'gene knockouts causing the {0} phenotype in mic from the IMPC Knockout Mouse Phenotypes dataset.', 'sets of gene knockouts causing phenotypes in mice from the IMPC Knockout Mouse Phenotypes dataset.', 'phenotypes of mice caused by {0} gene knockout from the IMPC Knockout Mouse Phenotypes dataset.', 0, 0, '2023-10-18', 'komp', 0, 109, 6, 1, 15, 3, 'association by various assays depending on phenotype', 'curated experimental data', 'high-throughput, data-driven', 'phenotypes', 0)

### Publication

In [ ]:
#(id, long_citation, short_citation, url, pmid, pubmed_url, first_author_last_name, first_author_initials, journal_abbreviation, year, title, volume, pages)
(147, 'Groza, T et al. (2023) The International Mouse Phenotyping Consortium: comprehensive knockout phenotyping underpinning the study of human disease. Nucleic Acids Res. 51:D1038-45', 'Groza, Nucleic Acids Res, 2023', 'dx.doi.org/10.1093/nar/gkac972', 36305825, 'https://pubmed.ncbi.nlm.nih.gov/36305825/', 'Groza', 'T', 'Nucleic Acids Res', 2023, 'The International Mouse Phenotyping Consortium: comprehensive knockout phenotyping underpinning the study of human disease', 51, 'D1038-45')
#(228, 148, 147)

### Genes

In [ ]:
index = 57552
genes = pd.read_csv('../../tables/gene.csv')
genelist = genes['symbol'].tolist()
genefks = genes.set_index('symbol')['id'].to_dict()
geneinfo = pd.read_csv('../../tables/gene_info', sep='\t').drop_duplicates('Symbol').set_index('Symbol').get(['GeneID', 'description'])
for gene in komp['Gene Symbol'].apply(str.upper).unique():
    if gene not in genelist:
        print((index, gene, geneinfo.loc[gene, 'GeneID'], geneinfo.loc[gene, 'description'], 'https://ncbi.nlm.nih.gov/gene/'+str(geneinfo.loc[gene, 'GeneID'])), end=',\n')
        genefks[gene] = index
        index += 1

### Gene Sets

In [ ]:
index = 134900000
genesetfks = {}
attributefks = pd.read_csv('../../tables/attribute.csv').set_index('name_from_naming_authority')['id'].to_dict()
for phenotype in komp['Phenotype Hits'].unique():
    print((index, phenotype, phenotypes.loc[phenotype, 0], phenotypes.loc[phenotype, 2], 148, 15, attributefks[phenotype]), end=',\n')
    genesetfks[phenotype] = index
    index += 1

### Associations

In [ ]:
associations = komp.copy()
associations['Gene Symbol'] = associations['Gene Symbol'].apply(str.upper).map(genefks)
associations['Phenotype Hits'] = associations['Phenotype Hits'].map(genesetfks)
associations['Threshold'] = 1
associations.columns = ['gene_fk', 'gene_set_fk', 'threshold_value']
associations.index += 30000000
associations.to_csv('../../harmonizome-update/komp.csv')
associations

## Create Downloads

In [ ]:
output_path = 'downloads/'
komp.columns = ['Gene', 'Phenotype']

### Gene-Attribute Binary Matrix

In [ ]:
binarymatrix = pd.crosstab(komp['Gene'], komp['Phenotype'])
binarymatrixT = binarymatrix.T
binarymatrix.to_csv(output_path+'gene_attribute_matrix.txt.gz', sep='\t', compression='gzip')
binarymatrix

### Gene-Attribute Edge List

In [ ]:
edgelist = komp.copy()
edgelist['Gene ID'] = edgelist['Gene'].apply(lambda x: geneinfo.loc[x,'GeneID'])
edgelist['Phenotype ID'] = edgelist['Phenotype'].apply(lambda x: phenotypes.loc[x, 0])
edgelist = edgelist.get(['Gene', 'Gene ID', 'Phenotype', 'Phenotype ID'])
edgelist['Threshold'] = 1
edgelist.to_csv(output_path+'gene_attribute_edges.txt.gz', sep='\t', compression='gzip')
edgelist

### Gene List

In [ ]:
geneslist = edgelist.get(['Gene', 'Gene ID']).drop_duplicates().reset_index(drop=True)
geneslist.to_csv(output_path+'gene_list_terms.txt.gz', sep='\t', compression='gzip')
geneslist

### Attribute List

In [ ]:
attributeslist = edgelist.get(['Phenotype', 'Phenotype ID']).drop_duplicates().reset_index(drop=True)
attributeslist.to_csv(output_path+'attribute_list_entries.txt.gz', sep='\t', compression='gzip')
attributeslist

### Gene Set Library

In [ ]:
with open(output_path+'gene_set_library_crisp.gmt', 'w') as f:
    arr = binarymatrix.reset_index(drop=True).to_numpy(dtype=np.int_)
    attributes = binarymatrix.columns

    w, h = arr.shape
    for i in tqdm(range(h)):
        if len([*binarymatrix.index[arr[:, i] == 1]])>= 5:
            print(attributes[i], *binarymatrix.index[arr[:, i] == 1], sep='\t', end='\n', file=f)

### Attribute Set Library

In [ ]:
with open(output_path+'attribute_set_library_crisp.gmt', 'w') as f:
    arr = binarymatrixT.reset_index(drop=True).to_numpy(dtype=np.int_)
    genes = binarymatrixT.columns

    w, h = arr.shape
    for i in tqdm(range(h)):
        if len([*binarymatrixT.index[arr[:, i] == 1]])>= 5:
            print(genes[i], *binarymatrixT.index[arr[:, i] == 1], sep='\t', end='\n', file=f)

### Gene Similarity Matrix

In [ ]:
gene_similarity_matrix = dist.pdist(binarymatrix.to_numpy(dtype=np.int_), 'cosine')
gene_similarity_matrix = dist.squareform(gene_similarity_matrix)
gene_similarity_matrix = 1 - gene_similarity_matrix

gene_similarity_matrix = pd.DataFrame(data=gene_similarity_matrix, index=binarymatrix.index, columns=binarymatrix.index)
gene_similarity_matrix.index.name = None
gene_similarity_matrix.columns.name = None
gene_similarity_matrix.to_csv(output_path+'gene_similarity_matrix_cosine.txt.gz', sep='\t', compression='gzip')
gene_similarity_matrix

### Attribute Similarity Matrix

In [ ]:
attribute_similarity_matrix = dist.pdist(binarymatrixT.to_numpy(dtype=np.int_), 'cosine')
attribute_similarity_matrix = dist.squareform(attribute_similarity_matrix)
attribute_similarity_matrix = 1 - attribute_similarity_matrix

attribute_similarity_matrix = pd.DataFrame(data=attribute_similarity_matrix, index=binarymatrixT.index, columns=binarymatrixT.index)
attribute_similarity_matrix.index.name = None
attribute_similarity_matrix.columns.name = None
attribute_similarity_matrix.to_csv(output_path+'attribute_similarity_matrix_cosine.txt.gz', sep='\t', compression='gzip')
attribute_similarity_matrix

### Knowledge Graph Serialization

In [ ]:
nodes = {}
edges = []

for gene in geneslist.index:
    gene = geneslist.loc[gene]
    nodes[int(gene['Gene ID'])] = {
        "type":"gene",
        "properties": {
            "id":int(gene['Gene ID']),
            "label":gene['Gene']
        }}

for phenotype in attributeslist.index:
    phenotype = attributeslist.loc[phenotype]
    nodes[phenotype['Phenotype ID']] = {
        "type":"phenotype",
        "properties": {
            "label":phenotype['Phenotype'],
            "id":phenotype['Phenotype ID']
        }}

for edge in edgelist.index:
    edge = edgelist.loc[edge]
    edges.append({
        "source": int(edge['Gene ID']),
        "relation": "has phenotype",
        "target": edge['Phenotype ID'],
        "properties":{
            "id":str(edge['Gene ID'])+":"+edge['Phenotype ID'],
            "source_id":int(edge['Gene ID']),
            "source_label":edge['Gene'],
            "target_id":edge['Phenotype ID'],
            "target_label":edge['Phenotype'],
            "directed":True,
            "threshold":1
        }})

#### RDF

In [ ]:
with open(output_path+'kg_serializations/komp.rdf', 'w') as f:
    print('@prefix gene: ncbi.nlm.nih.gov/gene/', file=f)
    print('@prefix RO: purl.obolibrary.org/RO_', file=f)
    print('@prefix MP: informatics.jax.org/vocab/mp_ontology/MP:', file=f)

    print('', file=f)
    for edge in edges:
        print('gene:'+str(edge['properties']['source_id']), 'RO:0002200', 'MP:'+edge['properties']['target_id'], end=' .\n', file=f)

#### JSON

In [ ]:
with open(output_path+'kg_serializations/komp.json', 'w') as f:
    serial = json.dump(
        {
            "Version":"1", 
            "nodes": nodes,
            "edges": edges
        }, indent=4, fp=f)

#### TSV

In [ ]:
nodeframe = pd.DataFrame(nodes).T
nodeframe['id'] = nodeframe['properties'].apply(lambda x: x['id'])
nodeframe['label'] = nodeframe['properties'].apply(lambda x: x['label'])
nodeframe['namespace'] = nodeframe['type'].apply(lambda x: {'gene':'NCBI Entrez', 'phenotype':'MPO'}[x])
nodeframe = nodeframe.get(['namespace', 'id', 'label']).reset_index(drop=True)
nodeframe.to_csv(output_path+'kg_serializations/komp_tsv/nodes.tsv', sep='\t')
nodeframe

In [ ]:
edgeframe = pd.DataFrame(edges)
edgeframe['threshold'] = edgeframe['properties'].apply(lambda x: x['threshold'])
edgeframe = edgeframe.get(['source', 'relation', 'target', 'threshold'])
edgeframe.to_csv(output_path+'kg_serializations/komp_tsv/edges.tsv', sep='\t')
edgeframe

## Create Visualizations

### Gene-Attribute Clustered Heatmap

In [ ]:
sns.clustermap(binarymatrix, cmap='seismic', center=0, figsize=(50, 50))

### Gene-Gene Similarity Clustered Heatmap

In [ ]:
sns.clustermap(gene_similarity_matrix, cmap='seismic', center=0)

### Attribute-Attribute Similarity Clustered Heatmap

In [ ]:
sns.clustermap(attribute_similarity_matrix, cmap='seismic', center=0)

### UMAP

In [ ]:
def load_gmt(file):
    gmt = OrderedDict()
    for line in file:
        term, *geneset = line.strip().split('\t')
        gmt[term] = ' '.join(set(geneset))
    return gmt

In [ ]:
libdict = load_gmt(open(output_path+'/gene_set_library_crisp.gmt', 'r'))
scatterdir = 'images/'

In [ ]:
def process_scatterplot(libdict, nneighbors=30, mindist=0.1, spread=1.0, maxdf=1.0, mindf=1):
    print("\tTF-IDF vectorizing gene set data...")
    vec = TfidfVectorizer(max_df=maxdf, min_df=mindf)
    X = vec.fit_transform(libdict.values())
    print(X.shape)
    adata = anndata.AnnData(X)
    adata.obs.index = libdict.keys()

    print("\tPerforming Leiden clustering...")
    ### the n_neighbors and min_dist parameters can be altered
    sc.pp.neighbors(adata, n_neighbors=nneighbors)
    sc.tl.leiden(adata, resolution=1.0)
    sc.tl.umap(adata, min_dist=mindist, spread=spread, random_state=42)

    new_order = adata.obs.sort_values(by='leiden').index.tolist()
    adata = adata[new_order, :]
    adata.obs['leiden'] = 'Cluster ' + adata.obs['leiden'].astype('object')

    df = pd.DataFrame(adata.obsm['X_umap'])
    df.columns = ['x', 'y']

    df['cluster'] = adata.obs['leiden'].values
    df['term'] = adata.obs.index
    df['genes'] = [libdict[l] for l in df['term']]

    return df

In [ ]:
def get_scatter_colors(df):
    clusters = pd.unique(df['cluster']).tolist()
    colors = list(Category20[20])[::2] + list(Category20[20])[1::2]
    color_mapper = {clusters[i]: colors[i % 20] for i in range(len(clusters))}
    return color_mapper

def get_scatterplot(scatterdf):
    df = scatterdf.copy()
    color_mapper = get_scatter_colors(df)
    df['color'] = df['cluster'].apply(lambda x: color_mapper[x])

    hover_emb = HoverTool(name="df", tooltips="""
        <div style="margin: 10">
            <div style="margin: 0 auto; width:300px;">
                <span style="font-size: 12px; font-weight: bold;">Gene Set:</span>
                <span style="font-size: 12px">@gene_set</span>
            <div style="margin: 0 auto; width:300px;">
                <span style="font-size: 12px; font-weight: bold;">Coordinates:</span>
                <span style="font-size: 12px">(@x,@y)</span>
            <div style="margin: 0 auto; width:300px;">
                <span style="font-size: 12px; font-weight: bold;">Cluster:</span>
                <span style="font-size: 12px">@cluster</span>
            </div>
        </div>
    """)
    tools_emb = [hover_emb, 'pan', 'wheel_zoom', 'reset', 'save']

    plot_emb = figure(
        width=1000, 
        height=700, 
        tools=tools_emb
    )

    source = ColumnDataSource(
        data=dict(
            x = df['x'],
            y = df['y'],
            gene_set = df['term'],
            cluster = df['cluster'],
            colors = df['color'],
            label = df['cluster']
        )
    )

    # hide axis labels and grid lines
    plot_emb.xaxis.major_tick_line_color = None
    plot_emb.xaxis.minor_tick_line_color = None
    plot_emb.yaxis.major_tick_line_color = None
    plot_emb.yaxis.minor_tick_line_color = None
    plot_emb.xaxis.major_label_text_font_size = '0pt'
    plot_emb.yaxis.major_label_text_font_size = '0pt' 

    plot_emb.output_backend = "svg"    
    
    plot_emb.title = 'Gene Sets in IMPC Knockout Mouse Phenotype Library'
    plot_emb.xaxis.axis_label = "UMAP_1"
    plot_emb.yaxis.axis_label = "UMAP_2"
    plot_emb.xaxis.axis_label_text_font_style = 'normal'
    plot_emb.xaxis.axis_label_text_font_size = '18px'
    plot_emb.yaxis.axis_label_text_font_size = '18px'
    plot_emb.yaxis.axis_label_text_font_style = 'normal'
    plot_emb.title.align = 'center'
    plot_emb.title.text_font_size = '18px'
    
    s = plot_emb.scatter(
        'x', 
        'y', 
        size = 4, 
        source = source, 
        color = 'colors'
    )
    
    return plot_emb

In [ ]:
## defaults: nneighbors=30, mindist=0.1, spread=1.0, maxdf=1.0, mindf=1
scatter_df = process_scatterplot(libdict, nneighbors=30,mindist=0.01
     ,spread=6.5, 
     maxdf=0.5, 
     mindf=15
)

# Display Scatter Plot
plot = get_scatterplot(scatter_df)
output_notebook()
show(plot)

In [ ]:
output_file(filename=f"{scatterdir}/umap.html", title = 'Gene Sets in IMCP Knockout Mouse Phenotype Library')
save(plot)